### The Moore-Penrose Pseudoinverse

---

Let's calculate the pseudoinverse $A^+$ of some matrix $A$ using the formula from the slides: 

$A^+ = VD^+U^T$

Where:
* $U, D$ and $V$ are SVD of $A$
* $D^+$ = ($D$ with reciprocal of all-non zero elements)$^T$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
A = np.array([[-1, 2], [3, -2], [5, 7]])
A

array([[-1,  2],
       [ 3, -2],
       [ 5,  7]])

As shown earlier the SVD methods return $U, d$ and $V^T$

In [3]:
U, d, VT = np.linalg.svd(A)
U, d, VT

(array([[ 0.12708324,  0.47409506,  0.87125411],
        [ 0.00164602, -0.87847553,  0.47778451],
        [ 0.99189069, -0.0592843 , -0.11241989]]),
 array([8.66918448, 4.10429538]),
 array([[ 0.55798885,  0.82984845],
        [-0.82984845,  0.55798885]]))

In [4]:
D = np.diag(d)
D

array([[8.66918448, 0.        ],
       [0.        , 4.10429538]])

In [5]:
1/8.66918448


0.11535110393682613

In [6]:
1/4.10429538

0.24364718116365203

We can do it manually like shown above but matrix is diagonal matrix so we can do it by inverting D

In [7]:
Dinv = np.linalg.inv(D)
Dinv

array([[0.1153511 , 0.        ],
       [0.        , 0.24364718]])

$D^+$ must have the same dimensions as $A^T$ in order for $VD^+U^T$ matrix multiplication to be possible: 

In [8]:
Dplus = np.concatenate((Dinv, np.zeros((1, 2)).T), axis=1)
Dplus

array([[0.1153511 , 0.        , 0.        ],
       [0.        , 0.24364718, 0.        ]])

bold text(Recall $D$ must have the same dimensions as $A$ for SVD's $UDV^T$, but for MPP $U$ and $V$ have swapped sides around the diagonal matrix.)

Now we have everything we need to calculate $A^+$ with $VD^+U^T$: 

In [9]:
Aplus = np.dot(VT.T, np.dot(Dplus, U.T))
Aplus

array([[-0.08767773,  0.17772512,  0.07582938],
       [ 0.07661927, -0.1192733 ,  0.08688784]])

Working out this derivation is helpful for understanding how Moore-Penrose pseudoinverses work, but unsurprisingly NumPy is loaded with an existing method `pinv()`: 

In [ ]:
np.linalg.pinv(A)

array([[-0.08767773,  0.17772512,  0.07582938],
       [ 0.07661927, -0.1192733 ,  0.08688784]])

**Exercise** 

Use the `torch.svd()` method to calculate the pseudoinverse of `A_p`, confirming that your result matches the output of `torch.pinverse(A_p)`:

In [11]:
A_p = torch.tensor([[-1, 2], [3, -2], [5, 7.]])
A_p

tensor([[-1.,  2.],
        [ 3., -2.],
        [ 5.,  7.]])

In [12]:
torch.pinverse(A_p)

tensor([[-0.0877,  0.1777,  0.0758],
        [ 0.0766, -0.1193,  0.0869]])

In [14]:
Ut, dt, Vt = torch.linalg.svd(A_p)
Ut, dt, Vt

(tensor([[ 0.1271,  0.4741,  0.8713],
         [ 0.0016, -0.8785,  0.4778],
         [ 0.9919, -0.0593, -0.1124]]),
 tensor([8.6692, 4.1043]),
 tensor([[ 0.5580,  0.8298],
         [-0.8298,  0.5580]]))

In [15]:
Dt = torch.diag(dt)
Dt

tensor([[8.6692, 0.0000],
        [0.0000, 4.1043]])

In [16]:
Dt_inv = torch.linalg.inv(Dt)
Dt_inv

tensor([[0.1154, 0.0000],
        [0.0000, 0.2436]])

In [24]:
Dt_plus =torch.cat((Dt_inv, torch.zeros((1, 2)).transpose(0, 1)), dim=1)
Dt_plus

tensor([[0.1154, 0.0000, 0.0000],
        [0.0000, 0.2436, 0.0000]])

In [30]:
Ap_plus = torch.linalg.matmul(Vt.T, torch.linalg.matmul(Dt_plus, Ut.T))
Ap_plus

tensor([[-0.0877,  0.1777,  0.0758],
        [ 0.0766, -0.1193,  0.0869]])

In [33]:
torch.linalg.pinv(A_p)

tensor([[-0.0877,  0.1777,  0.0758],
        [ 0.0766, -0.1193,  0.0869]])